In [1]:
!pip install langchain
!pip install langchain-openai
!pip install langchain-core

In [2]:
# uncomment this if running locally
# from dotenv import load_dotenv

# load_dotenv()

# Or if you are in Colab, uncoment below and add your api key
import os
os.environ["OPENAI_API_KEY"] = "your-api-key"

# Common Sense Agent

1. Model (large language model like ChatGPT, or Claude)
2. Prompt (system message and the user input)
3. Tool calling

In [3]:
from langchain_openai import ChatOpenAI
import os
# os.environ["OPENAI_API_KEY"] = "your openai api key"
llm = ChatOpenAI(model="gpt-4o", temperature=0)

llm.invoke("Hi")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_18cc0f1fa0', 'finish_reason': 'stop', 'logprobs': None}, id='run-4c866c8f-958e-45d7-bb89-50286c950997-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17})

In [4]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Answer this question: {question} in bullet points. Output:\n")

In [5]:
chain = prompt | llm

chain

ChatPromptTemplate(input_variables=['question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='Answer this question: {question} in bullet points. Output:\n'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10637abd0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x106573790>, model_name='gpt-4o', temperature=0.0, openai_api_key=SecretStr('**********'), openai_api_base='https://api.openai.com/v1', openai_proxy='')

In [6]:
type(chain)

langchain_core.runnables.base.RunnableSequence

In [7]:
output = chain.invoke("Write down names of cats.")
output

AIMessage(content='- Whiskers\n- Mittens\n- Shadow\n- Luna\n- Simba\n- Oliver\n- Bella\n- Charlie\n- Max\n- Daisy', response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 23, 'total_tokens': 55}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_c4e5b6fa31', 'finish_reason': 'stop', 'logprobs': None}, id='run-f0a8dc85-f6f7-4e25-9466-e06029c5a425-0', usage_metadata={'input_tokens': 23, 'output_tokens': 32, 'total_tokens': 55})

In [8]:
from IPython.display import Markdown

Markdown(output.content)

- Whiskers
- Mittens
- Shadow
- Luna
- Simba
- Oliver
- Bella
- Charlie
- Max
- Daisy

In [9]:
from langchain_core.tools import tool
from serpapi import GoogleSearch

serpapi_params = {
    "engine": "google",
    "api_key": os.environ["SERPAPI_KEY"]
}

query = "Cool buildings in Paris"
search = GoogleSearch({**serpapi_params, "q":query, "n": 3})
search.get_dict()["organic_results"]

[{'position': 1,
  'title': '25 Must-See Paris Landmarks',
  'link': 'https://www.architecturaldigest.com/gallery/paris-architectural-landmarks',
  'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.architecturaldigest.com/gallery/paris-architectural-landmarks&ved=2ahUKEwjD2czuxrCHAxWaEFkFHf6ZDOUQFnoECBsQAQ',
  'displayed_link': 'https://www.architecturaldigest.com › ... › travel',
  'thumbnail': 'https://serpapi.com/searches/6699059bf8f5f77c7a8ee8c8/images/b82971cf6568ad27e2fa13ba6a1ef50fcedef7e58fe760b01529225406d3a2a2.jpeg',
  'favicon': 'https://serpapi.com/searches/6699059bf8f5f77c7a8ee8c8/images/b82971cf6568ad27e2fa13ba6a1ef50f8f513dcefb7186a53f3be813729d0233.png',
  'date': 'Feb 18, 2016',
  'snippet': 'Located on the Île de la Cité, a thin island in the Seine, Notre Dame de Paris is perhaps the most famous cathedral in the world. Characterized ...',
  'snippet_highlighted_words': ['Paris', 'famous'],
  'source': 'Architectural Digest

In [10]:
results = search.get_dict()["organic_results"]
contexts = "\n---\n".join(
       ["\n".join([x["title"], x["snippet"], x["link"]]) for x in results]
   )
contexts

"25 Must-See Paris Landmarks\nLocated on the Île de la Cité, a thin island in the Seine, Notre Dame de Paris is perhaps the most famous cathedral in the world. Characterized ...\nhttps://www.architecturaldigest.com/gallery/paris-architectural-landmarks\n---\nTHE 10 BEST Paris Architectural Buildings (Updated 2024)\nArchitectural Buildings in Paris · 1. Louvre Museum · 2. Cathédrale Notre-Dame de Paris · 3. Arc de Triomphe · 4. Palais Garnier · 5. Sainte- ...\nhttps://www.tripadvisor.com/Attractions-g187147-Activities-c47-t3-Paris_Ile_de_France.html\n---\nParis City Guide: 23 Places Every Architect Must Visit\n1. Centre Georges Pompidou ... Save this picture! ... Description: This is one of the most iconic buildings in Paris and houses the Musée National d ...\nhttps://www.archdaily.com/922278/23-places-in-paris-every-architect-must-visit\n---\n22 of the Most Famous Buildings in Paris You Must See\nMost Famous Buildings in Paris You Must See · 1. The Louvre Museum · 2. Notre-Dame Cathed

In [11]:
@tool
def web_search(query: str) -> str:
    """Finds general knowledge information using Google search. Can also be used
    to augment more 'general' knowledge to a previous specialist query."""
    search = GoogleSearch({**serpapi_params, "q":query, "n": 3})
    results = search.get_dict()["organic_results"]
    contexts = "\n---\n".join(
         ["\n".join([x["title"], x["snippet"], x["link"]]) for x in results]
        )
    
    return contexts

web_search.invoke("Cool buildings in Paris")

"25 Must-See Paris Landmarks\nLocated on the Île de la Cité, a thin island in the Seine, Notre Dame de Paris is perhaps the most famous cathedral in the world. Characterized ...\nhttps://www.architecturaldigest.com/gallery/paris-architectural-landmarks\n---\nTHE 10 BEST Paris Architectural Buildings (Updated 2024)\nArchitectural Buildings in Paris · 1. Louvre Museum · 2. Cathédrale Notre-Dame de Paris · 3. Arc de Triomphe · 4. Palais Garnier · 5. Sainte- ...\nhttps://www.tripadvisor.com/Attractions-g187147-Activities-c47-t3-Paris_Ile_de_France.html\n---\nParis City Guide: 23 Places Every Architect Must Visit\n1. Centre Georges Pompidou ... Save this picture! ... Description: This is one of the most iconic buildings in Paris and houses the Musée National d ...\nhttps://www.archdaily.com/922278/23-places-in-paris-every-architect-must-visit\n---\n22 of the Most Famous Buildings in Paris You Must See\nMost Famous Buildings in Paris You Must See · 1. The Louvre Museum · 2. Notre-Dame Cathed

In [12]:
llm_with_tool = llm.bind_tools([web_search])

research_chain = prompt | llm_with_tool 

research_chain.invoke("What are the cool buildings in Paris?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_UiQaEuRqmg8kRlbaRX4N1WpH', 'function': {'arguments': '{"query":"cool buildings in Paris"}', 'name': 'web_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 82, 'total_tokens': 99}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_c4e5b6fa31', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3d6c2934-e3d1-4161-9b38-67ae33033522-0', tool_calls=[{'name': 'web_search', 'args': {'query': 'cool buildings in Paris'}, 'id': 'call_UiQaEuRqmg8kRlbaRX4N1WpH', 'type': 'tool_call'}], usage_metadata={'input_tokens': 82, 'output_tokens': 17, 'total_tokens': 99})

In [13]:
sample_search_result = research_chain.invoke("What are the cool buildings in Paris?")
sample_search_result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_lOGEADkjce7NSnEeIR5bTnxb', 'function': {'arguments': '{"query":"cool buildings in Paris"}', 'name': 'web_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 82, 'total_tokens': 99}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_c4e5b6fa31', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-758f5fca-53d1-48eb-97c2-750eae67dfbf-0', tool_calls=[{'name': 'web_search', 'args': {'query': 'cool buildings in Paris'}, 'id': 'call_lOGEADkjce7NSnEeIR5bTnxb', 'type': 'tool_call'}], usage_metadata={'input_tokens': 82, 'output_tokens': 17, 'total_tokens': 99})

In [14]:
sample_search_result.tool_calls

[{'name': 'web_search',
  'args': {'query': 'cool buildings in Paris'},
  'id': 'call_lOGEADkjce7NSnEeIR5bTnxb',
  'type': 'tool_call'}]

In [15]:
sample_search_result.tool_calls[0]['args']

{'query': 'cool buildings in Paris'}

In [16]:
research_chain = prompt | llm_with_tool | (lambda x: x.tool_calls[0]['args'])

research_chain.invoke("What are the cool buildings in Paris?")

{'query': 'cool buildings in Paris'}

In [17]:
research_chain = prompt | llm_with_tool | (lambda x: x.tool_calls[0]['args']) | web_search

research_chain.invoke("What are the cool buildings in Paris?")

"25 Must-See Paris Landmarks\nLocated on the Île de la Cité, a thin island in the Seine, Notre Dame de Paris is perhaps the most famous cathedral in the world. Characterized ...\nhttps://www.architecturaldigest.com/gallery/paris-architectural-landmarks\n---\nTHE 10 BEST Paris Architectural Buildings (Updated 2024)\nArchitectural Buildings in Paris · 1. Louvre Museum · 2. Cathédrale Notre-Dame de Paris · 3. Arc de Triomphe · 4. Palais Garnier · 5. Sainte- ...\nhttps://www.tripadvisor.com/Attractions-g187147-Activities-c47-t3-Paris_Ile_de_France.html\n---\nParis City Guide: 23 Places Every Architect Must Visit\n1. Centre Georges Pompidou ... Save this picture! ... Description: This is one of the most iconic buildings in Paris and houses the Musée National d ...\nhttps://www.archdaily.com/922278/23-places-in-paris-every-architect-must-visit\n---\n22 of the Most Famous Buildings in Paris You Must See\nMost Famous Buildings in Paris You Must See · 1. The Louvre Museum · 2. Notre-Dame Cathed

In [18]:
def fake_agent(query: str) -> str:
    search_result = research_chain.invoke("What are the cool buildings in Paris?")
    fake_agent = llm.invoke(f"The user query was: {query}. The web search result for\
        that is: {search_result}. Give a summarized answer:")
    
    return fake_agent.content

fake_agent("What are the cool buildings in Paris?")

'Paris is home to numerous iconic and beautiful buildings. Here are some must-see architectural landmarks:\n\n1. **Louvre Museum** - Famous for its art collection and the glass pyramid entrance.\n2. **Notre-Dame Cathedral** - Renowned Gothic cathedral located on Île de la Cité.\n3. **Arc de Triomphe** - Monument honoring those who fought and died for France.\n4. **Palais Garnier** - Opulent opera house.\n5. **Sainte-Chapelle** - Known for its stunning stained glass windows.\n6. **Centre Georges Pompidou** - Modern art museum with a unique architectural design.\n7. **Basilica of the Sacré-Cœur** - White-domed basilica located at the highest point in the city.\n8. **Panthéon** - Mausoleum containing the remains of distinguished French citizens.\n9. **Fondation Louis Vuitton** - Contemporary art museum with striking modern architecture.\n10. **Villa Savoye** - Iconic modernist villa designed by Le Corbusier.\n\nThese buildings showcase a mix of historical and modern architecture, making P

In [27]:
from langchain_core.runnables import RunnablePassthrough
from langchain.agents.format_scratchpad.tools import format_to_tool_messages
from langchain.agents.output_parsers.tools import ToolsAgentOutputParser

prompt = ChatPromptTemplate.from_messages(
    [
    ("system", "You are a helpful assistant"),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
    ]
                                          )

llm = ChatOpenAI(model="gpt-4o", temperature=0)

tools = [web_search]

llm_with_tools = llm.bind_tools(tools)

research_agent = (
                 RunnablePassthrough.assign(
                    agent_scratchpad=lambda x: format_to_tool_messages(x['intermediate_steps'])
                    )
) | prompt | llm_with_tools | ToolsAgentOutputParser()

research_agent.invoke({"input": "What are the cool buildings in Paris?", "intermediate_steps": []})

[ToolAgentAction(tool='web_search', tool_input={'query': 'cool buildings in Paris'}, log="\nInvoking: `web_search` with `{'query': 'cool buildings in Paris'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_2CITYbhQK01ngDXpSrIhW65b', 'function': {'arguments': '{"query":"cool buildings in Paris"}', 'name': 'web_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 87, 'total_tokens': 104}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_c4e5b6fa31', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-30377ed5-2eaa-45fb-9cb8-b384ac3f1c68-0', tool_calls=[{'name': 'web_search', 'args': {'query': 'cool buildings in Paris'}, 'id': 'call_2CITYbhQK01ngDXpSrIhW65b'}])], tool_call_id='call_2CITYbhQK01ngDXpSrIhW65b')]

In [20]:
prompt

ChatPromptTemplate(input_variables=['input'], optional_variables=['agent_scratchpad', 'chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': [], 'agent_scratchpad': []}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromp

In [21]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=research_agent,tools=tools)
agent_executor.invoke({'input': 'What are cool buildings in Alicante Spain?'})

{'input': 'What are cool buildings in Alicante Spain?',
 'output': "Alicante, Spain, is home to several cool and notable buildings. Here are some of the highlights:\n\n1. **Castle of Santa Bárbara**: A historic castle offering panoramic views of the city and the Mediterranean Sea.\n2. **Casa Carbonell**: A remarkable architectural gem and historic building that stands as a testament to Alicante's rich heritage.\n3. **The Town Hall (Ayuntamiento de Alicante)**: An impressive baroque building located in the heart of the city.\n4. **The Central Market (Mercado Central)**: A bustling market housed in a beautiful early 20th-century building.\n5. **Teatro Principal**: The main theater in Alicante, known for its neoclassical architecture.\n6. **Bullring (Plaza de Toros)**: A historic bullring that is still in use today for various events.\n7. **Diputacion Provincial de Alicante**: Another notable building with significant architectural value.\n8. **Archives Municipales**: A building that hous

Wwe can automate part of this process by using a built in method in LangChain
that abstracts away the chaining procedure.

In [22]:
from langchain.agents import create_tool_calling_agent
from langchain_openai import ChatOpenAI
from langchain import hub

llm = ChatOpenAI(model="gpt-4o", temperature=0)

tools = [web_search]

prompt = hub.pull("hwchase17/openai-tools-agent")

agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools)

agent_executor.invoke({'input': 'What are cool buildings in Alicante Spain?'})

{'input': 'What are cool buildings in Alicante Spain?',
 'output': "Alicante, Spain, is home to several cool and notable buildings. Here are some of the highlights:\n\n1. **Castle of Santa Bárbara**: A historic castle offering panoramic views of the city and the Mediterranean Sea.\n2. **Casa Carbonell**: A remarkable architectural gem and historic building that stands as a testament to Alicante's rich heritage.\n3. **The Town Hall (Ayuntamiento de Alicante)**: An impressive baroque building located in the heart of the city.\n4. **The Central Market (Mercado Central)**: A bustling market housed in a beautiful early 20th-century building.\n5. **Teatro Principal**: The main theater in Alicante, known for its neoclassical architecture.\n6. **Bullring (Plaza de Toros)**: A historic bullring that is still in use today for various events.\n7. **Diputacion Provincial de Alicante**: Another notable building with significant architectural value.\n8. **Archives Municipales**: A building that hous

In [23]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-tools-agent', 'lc_hub_commit_hash': 'c18672812789a3b9697656dd539edf0120285dcae36396d0b548ae42a4ed66f5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(in